Example of building a simple CNN for CIFAR10 dataset. 

In [ ]:
import zipfile
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!unzip "/content/drive/My Drive/EuroSAT-RGB.zip"

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import shutil

In [ ]:

#transforms train & test data 

transforms_train = transforms.Compose([
      
    transforms.Resize((64, 64)),   #must same as here
    transforms.RandomHorizontalFlip(), # data augmentation
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # normalization
])


transforms_test = transforms.Compose([
  
    transforms.Resize((64, 64)),   #must same as here
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


In [ ]:
# # Creating Train  / Test folders (One time use)
# this  runs once


import shutil
import os
import numpy as np

classes =[ '/River', '/SeaLake','/AnnualCrop','/Forest','/HerbaceousVegetation','/Highway','/Industrial','/Pasture','/PermanentCrop','/Residential']
root_dir = 'DataSet01'
print(len(classes))
index =0
while index+1 < len(classes):
  posCls = classes[index]
  negCls = classes[index+1]
  index+=2

  os.makedirs(root_dir +'/train' + posCls)
  os.makedirs(root_dir +'/train' + negCls)

  os.makedirs(root_dir +'/test' + posCls)
  os.makedirs(root_dir +'/test' + negCls)

  # Creating partitions of the data after shuffeling
  currentCls = posCls
  src = "EuroSAT-RGB/2750/"+currentCls # Folder to copy images from

  allFileNames = os.listdir(src)
  np.random.shuffle(allFileNames)
  train_FileNames, val_FileNames, test_FileNames = np.split(np.array(allFileNames),
                                                            [int(len(allFileNames)*0.7), int(len(allFileNames)*0.85)])


  train_FileNames = [src+'/'+ name for name in train_FileNames.tolist()]
  test_FileNames = [src+'/' + name for name in test_FileNames.tolist()]

# Copy-pasting images
  for name in train_FileNames:
      shutil.copy(name, root_dir+"/train"+currentCls)



  for name in test_FileNames:
      shutil.copy(name, root_dir+"/test"+currentCls)
  # Creating partitions of the data after shuffeling
  currentCls = negCls
  src = "EuroSAT-RGB/2750/"+currentCls # Folder to copy images from

  allFileNames = os.listdir(src)
  np.random.shuffle(allFileNames)
  # spliting every class with balancing them
  train_FileNames, test_FileNames = np.split(np.array(allFileNames),
                                                            [int(len(allFileNames)*0.7)])
  train_FileNames = [src+'/'+ name for name in train_FileNames.tolist()]
  test_FileNames = [src+'/' + name for name in test_FileNames.tolist()]

  for name in train_FileNames:
      shutil.copy(name, root_dir+"/train"+currentCls)

  for name in test_FileNames:
      shutil.copy(name, root_dir+"/test"+currentCls)

10


In [ ]:
# use GPU if avaliable
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [ ]:
''' 
We import the necessary libraries and modules for our code.
We use PyTorch as our deep learning framework, 
nn module for defining neural networks, 
optim module for optimization algorithms, 
and transforms module for data preprocessing.
'''


# Define the RESNET 18 architecture

# Create NN For common part
class NN_Class(nn.Module):
    
    def __init__(self, in_channels, out_channels, identity_downsample=None, stride=1):
        super(NN_Class, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()
        self.identity_downsample = identity_downsample
        
    def forward(self, x):
        identity = x
        # conv 1
        x = self.conv1(x)
        # batch normailztion 1
        x = self.bn1(x)
        # RELU
        x = self.relu(x)
        # conv 2
        x = self.conv2(x)
        # batch normalzaztion 2
        x = self.bn2(x)
        # identity matix
        # do down sampling
        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity)
        x += identity
        # RELU
        x = self.relu(x)
        return x



class Net(nn.Module):
    def __init__(self,image_channels, num_classes):
        """
        CNN Architecture: 2 convolutional layers, 1 max pool, 2 fully connected layer, 
        and final fully connected layer to classify 10 different classes
        """
        super(Net, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(image_channels, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        # Depp NN layers
        self.layer1 = self.create_layer(64, 64, stride=1)
        self.layer2 = self.create_layer(64, 128, stride=2)
        self.layer3 = self.create_layer(128, 256, stride=2)
        self.layer4 = self.create_layer(256, 512, stride=2)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

      
    def create_layer(self, in_channels, out_channels, stride):
        
        identity_downsample = None
        if stride != 1:
            identity_downsample = self.identity_downsample(in_channels, out_channels)
            
        return nn.Sequential(
            NN_Class(in_channels, out_channels, identity_downsample=identity_downsample, stride=stride), 
            NN_Class(out_channels, out_channels)
        )

    def forward(self, x):
        """
        Defines the forward pass of the network.
        """
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avgpool(x)
        x = x.view(x.shape[0], -1)
        x = self.fc(x)
        return x 
    def identity_downsample(self, in_channels, out_channels):
        
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=2, padding=1), 
            nn.BatchNorm2d(out_channels)
        )
      

# Load the dataset

"""
Creates a data transformation pipeline using the transforms.Compose function from the PyTorch library.

"""





train_dir = root_dir+"/train/"
test_dir = root_dir+"/test/"


train_dataset = ImageFolder(train_dir, transforms_train)
test_dataset = ImageFolder(test_dir, transforms_test)


train_dataloader = DataLoader(train_dataset, batch_size=12, shuffle=True, num_workers=8)
test_dataloader = DataLoader(test_dataset, batch_size=12, shuffle=False, num_workers=8)

# Train the network
model = Net(3, 10)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)
num_epochs=5

for epoch in range(num_epochs): #(loop for every epoch)
    print("Epoch {} running".format(epoch)) #(printing message)
    """ Training Phase """
    model.train()    #(training model)
    running_loss = 0.   #(set loss 0)
    running_corrects = 0 
    # load a batch data of images
    targets = []
    results = []
    i=0
    for inputs,labels in train_dataloader:
        i+=1
        outputs = model(inputs)        
        optimizer.zero_grad()
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        results.append(preds)
        targets.append(labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
    results =torch.cat(results, dim=0)
    targets = torch.cat(targets, dim=0)

    results = results.to('cpu').numpy().flatten()
    targets = targets.to('cpu').numpy().flatten()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = running_corrects / len(train_dataset) * 100.
    print(epoch, epoch_loss, epoch_acc)


Epoch 0 running


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


0 0.9387733889666815 tensor(66.3757)
Epoch 1 running
1 0.6314082072083912 tensor(77.6614)
Epoch 2 running
2 0.49854683318781473 tensor(82.4762)
Epoch 3 running
3 0.41817943485719816 tensor(85.5344)
Epoch 4 running
4 0.3446819563777674 tensor(88.2169)


In [ ]:
    print('Training acc', epoch_acc)

Training acc tensor(88.2169)


In [ ]:

    """ Test Phase """
    model.eval()
    with torch.no_grad():
        running_loss = 0.
        running_corrects = 0
        targets = []
        results = []
        for inputs, labels in test_dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            results.append(preds)
            targets.append(labels)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
        epoch_loss = running_loss / len(test_dataset)
        epoch_acc = running_corrects / len(test_dataset) * 100.
        results = torch.cat(results, dim=0)
        targets =torch.cat(targets, dim=0)
        results = results.to('cpu').numpy().flatten()
        targets = targets.to('cpu').numpy().flatten()
       
        print("Test Accu",epoch_acc)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Test Accu tensor(88.2305)
